In [3]:
import numpy as np 
import pandas as pd 
import findspark

In [4]:
#importar libreria e iniciar sesior
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder.appName('UCI enfermedades del corazon')\
        .getOrCreate()

In [5]:
#implementación del algoritmo de regresión logística que se encuentra en la biblioteca 
from pyspark.ml.classification import LogisticRegression

In [7]:
#leer el CSV
corazon = spark.read.csv(r'C:\Users\rc_sh\OneDrive\Escritorio\spark\Analítica+de+datos+con+PySpark\data\heart.csv', 
                       inferSchema = True, 
                       header = True)
corazon.show(6)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 6 rows



In [8]:
#conversion a Pandas
corazon.toPandas().head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [9]:
#PROCESAR LOS DATOS

In [11]:
#Importar librerias a usar
#crear objetos de vector que contienen las características de cada instancia en un conjunto de datos
from pyspark.ml.linalg import Vectors
#transformación que combina múltiples columnas de características en un solo vector de características
from pyspark.ml.feature import VectorAssembler

In [13]:
#mostrar  las columnas
corazon.columns

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target']

In [14]:
# inputCols como entrada para algoritmos para los siguiente nueva columna outpult features
assembler = VectorAssembler(
                            inputCols=['age',
                            'sex',
                            'cp',
                            'trestbps',
                            'chol',
                            'fbs',
                            'restecg',
                            'thalach',
                            'exang',
                            'oldpeak',
                            'slope',
                            'ca',
                            'thal'],
                            outputCol="features")

In [15]:
#nuevo DataFrame que incluye todas las columnas originales de corazon y, una nueva columna llamada features
output = assembler.transform(corazon)
output.show(6)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|[63.0,1.0,3.0,145...|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|[37.0,1.0,2.0,130...|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|[41.0,0.0,1.0,130...|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|[56.0,1.0,1.0,120...|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|[57.0,0.0,0.0,120...|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|[57.0,1.0,0.0,140...|
+---+---+---+--------+----+---+-------+-------+-----+--

In [16]:
#seleccionar columnas y crear nuevo dataFrame
final_data = output.select("features",'target')

In [17]:
#ENTREANAR DATOS

In [18]:
#ividir un DataFrame en dos conjuntos de datos primer elemento 70% y el segundo elemento 30% de la prueba
train, test = final_data.randomSplit([0.7,0.3])

In [19]:
#modelo de regresion, parametros principales, labelCol y featruesCol
lr = LogisticRegression(labelCol="target",
                        featuresCol="features")


In [20]:
#ajustar el modelo
model = lr.fit(train)

In [27]:
#prediccion: 
prediccion=model.transform(train)
prediccion=model.transform(test)
prediccion.select("target","prediction").show()

+------+----------+
|target|prediction|
+------+----------+
|     0|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     0|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     0|       0.0|
|     1|       1.0|
|     0|       0.0|
|     0|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
+------+----------+
only showing top 20 rows



In [28]:
#EVALUACION

In [29]:
#Importar biblioteca para evaluar
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Configuracion de la evaluacion
#rawPredictionCol='rawPrediction': Especifica la columna que contiene las predicciones brutas del modelo.
#labelCol='target': Especifica la columna que contiene las etiquetas reales (la variable objetivo) en los datos de prueba.
evaluacion = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol='target')
#contiene los resultados de las predicciones del modelo sobre los datos de prueba.
prediccion.select("target","rawPrediction","prediction","probability").show()

+------+--------------------+----------+--------------------+
|target|       rawPrediction|prediction|         probability|
+------+--------------------+----------+--------------------+
|     0|[-1.1792252401565...|       1.0|[0.23519152871187...|
|     1|[-1.4597510990713...|       1.0|[0.18850539691376...|
|     1|[-5.1170848023076...|       1.0|[0.00595776193604...|
|     1|[-1.7491717358471...|       1.0|[0.14815169680399...|
|     1|[-0.6735548465885...|       1.0|[0.33770130874260...|
|     1|[-2.0888394108513...|       1.0|[0.11018631319190...|
|     1|[-3.7164205466069...|       1.0|[0.02374340737028...|
|     1|[-3.4825673121236...|       1.0|[0.02981233429690...|
|     0|[-3.0023630122985...|       1.0|[0.04731923426836...|
|     1|[-3.7517573244226...|       1.0|[0.02293795206635...|
|     1|[-4.8653628272865...|       1.0|[0.00765005612030...|
|     1|[-0.4502159335573...|       1.0|[0.38930942713949...|
|     0|[0.40856762014085...|       0.0|[0.60074437134865...|
|     1|

In [32]:
#para evaluar y mostrar los datos
print("El área bajo la curva ROC para el conjunto de entrenamiento es {}".format(evaluacion.evaluate(prediccion)))

print("El área bajo la curva ROC para el conjunto de prueba es {}".format(evaluacion.evaluate(prediccion)))

El área bajo la curva ROC para el conjunto de entrenamiento es 0.8984126984126982
El área bajo la curva ROC para el conjunto de prueba es 0.8984126984126982


In [ ]:
#El área bajo la curva ROC para el conjunto de entrenamiento es 0.8984126984126982:
el modelo tiene una capacidad del 89.84% para distinguir entre las clases positivas y negativas en el conjunto de datos de entrenamiento.
El área bajo la curva ROC para el conjunto de prueba es 0.8984126984126982:

Indica que el modelo tiene una capacidad del 89.84% para distinguir entre las clases positivas y negativas en el conjunto de datos de prueba.